In [ ]:
import os
import pickle
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from flask import Flask, render_template, request, jsonify



# Initialize Flask app
app = Flask(__name__)

# Define paths
MODEL_DIR = 'farm_practice_models'

# Define emotional aspects
emotional_aspects = {
    'motivation': ['motivat', 'inspir', 'drive', 'enthusias', 'passion', 'eager', 'excit'],
    'satisfaction': ['satisf', 'content', 'happy', 'enjoy', 'proud', 'fulfill', 'achiev'],
    'challenge': ['challeng', 'difficult', 'hard', 'struggl', 'frustrat', 'stress', 'overwhelm'],
    'learning': ['learn', 'knowledge', 'skill', 'understand', 'educat', 'growth', 'develop'],
    'social': ['team', 'collaborat', 'friend', 'interact', 'communit', 'connect', 'relation']
}

# Load models, tokenizer, and encoders
aspect_models = {}
aspect_encoders = {}

# Check if models directory exists
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR, exist_ok=True)
    print(f"Created model directory at {MODEL_DIR}. Please place your models here.")
else:
    # Load tokenizer if it exists
    tokenizer_path = os.path.join(MODEL_DIR, 'tokenizer.pkl')
    if os.path.exists(tokenizer_path):
        with open(tokenizer_path, 'rb') as f:
            tokenizer = pickle.load(f)
        print("Tokenizer loaded successfully")
    else:
        print(f"Tokenizer not found at {tokenizer_path}")
    
    # Load aspect models and encoders
    for aspect in emotional_aspects.keys():
        model_path = os.path.join(MODEL_DIR, f'{aspect}_model')
        encoder_path = os.path.join(MODEL_DIR, f'{aspect}_encoder.pkl')
        
        if os.path.exists(model_path) and os.path.exists(encoder_path):
            try:
                aspect_models[aspect] = load_model(model_path)
                with open(encoder_path, 'rb') as f:
                    aspect_encoders[aspect] = pickle.load(f)
                print(f"Loaded model and encoder for {aspect}")
            except Exception as e:
                print(f"Error loading {aspect} model: {str(e)}")
        else:
            print(f"Model or encoder for {aspect} not found")

# Text preprocessing functions
def clean_text(text):
    text = text.lower()  # convert to lowercase
    text = re.sub(f'[{string.punctuation}]', ' ', text)  # remove punctuation
    text = re.sub(r'\d+', '', text)  # remove numbers
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra whitespace
    return text

def preprocess_text(text):
    # Initialize NLTK components
    try:
        lemmatizer = WordNetLemmatizer()
        stop_words = set(stopwords.words('english'))
    except:
        import nltk
        nltk.download('punkt')
        nltk.download('stopwords')
        nltk.download('wordnet')
        lemmatizer = WordNetLemmatizer()
        stop_words = set(stopwords.words('english'))
    
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens)

# Analyze sentiment for new text
def analyze_farm_experience(text):
    results = {}
    
    # Clean and process text
    cleaned = clean_text(text)
    processed = preprocess_text(cleaned)
    
    # Extract overall sentiment using rule-based approach
    positive_words = ['enjoy', 'great', 'good', 'positive', 'happy', 'satisfied', 'exciting',
                      'love', 'wonderful', 'excellent', 'amazing', 'fantastic', 'terrific']
    negative_words = ['difficult', 'hard', 'challenging', 'frustrat', 'stress', 'negative',
                      'overwhelm', 'disappointment', 'dislike', 'hate', 'terrible', 'awful', 'poor']
    
    positive_count = sum(1 for word in positive_words if word in processed)
    negative_count = sum(1 for word in negative_words if word in processed)
    
    if positive_count > negative_count:
        results['overall_sentiment'] = 'positive'
    elif negative_count > positive_count:
        results['overall_sentiment'] = 'negative'
    else:
        results['overall_sentiment'] = 'neutral'
    
    # Set maximum sequence length for padding
    max_seq_length = 100
    aspects_present = {}
    
    for aspect, keywords in emotional_aspects.items():
        # Check if aspect is mentioned in the processed text
        is_present = any(re.search(r'\b' + keyword + r'[a-z]*\b', processed) for keyword in keywords)
        aspects_present[aspect] = is_present
        
        if is_present:
            # If the model, encoder, or tokenizer is missing, mark as not available
            if aspect not in aspect_models or aspect not in aspect_encoders or 'tokenizer' not in globals():
                results[aspect] = 'model not available'
            else:
                # Convert text to sequence and pad
                sequence = tokenizer.texts_to_sequences([processed])
                padded_sequence = pad_sequences(sequence, maxlen=max_seq_length, padding='post')
                
                # Make prediction and convert to class label
                prediction = aspect_models[aspect].predict(padded_sequence)[0]
                encoder = aspect_encoders[aspect]
                if len(encoder.classes_) == 2:
                    idx = 1 if prediction > 0.5 else 0
                else:
                    idx = np.argmax(prediction)
                results[aspect] = encoder.classes_[idx]
        else:
            results[aspect] = 'not_mentioned'
    
    # Include aspect presence information in results
    results['aspects_present'] = aspects_present
    return results

# Routes
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/analyze', methods=['POST'])
def analyze():
    if request.method == 'POST':
        text = request.form['experience_text']
        if not text:
            return jsonify({'error': 'Please enter some text to analyze'})
        
        try:

            results = analyze_farm_experience(text)
            return jsonify(results)
        except Exception as e:
            return jsonify({'error': f'Error analyzing text: {str(e)}'})

@app.route('/model_status')
def model_status():
    status = {
        'tokenizer_loaded': 'tokenizer' in globals(),
        'models_loaded': {aspect: aspect in aspect_models for aspect in emotional_aspects.keys()}
    }

    return jsonify(status)

# Run the app
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)



Tokenizer loaded successfully
Model or encoder for motivation not found
Model or encoder for satisfaction not found
Model or encoder for challenge not found
Model or encoder for learning not found
Model or encoder for social not found
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Jun/2025 23:23:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2025 23:23:05] "GET /model_status HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2025 23:23:06] "GET /favicon.ico HTTP/1.1" 404 -
